In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, cohen_kappa_score
from simpletransformers.ner import NERModel,NERArgs

## Load the Data

In [71]:
path_to_ner_data = 'DATA/SYNTHETIC/ner_labeled.xlsx'
data = pd.read_excel(path_to_ner_data, index_col=0, engine = 'openpyxl')

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15110 entries, 0 to 15109
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   testcase_id  15110 non-null  int64 
 1   word_num     15110 non-null  int64 
 2   word         15108 non-null  object
 3   label        15110 non-null  object
dtypes: int64(2), object(2)
memory usage: 590.2+ KB


In [73]:
data.fillna('NaN', inplace=True)

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15110 entries, 0 to 15109
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   testcase_id  15110 non-null  int64 
 1   word_num     15110 non-null  int64 
 2   word         15110 non-null  object
 3   label        15110 non-null  object
dtypes: int64(2), object(2)
memory usage: 590.2+ KB


In [75]:
data = data[data['word'].str.len() > 1]

In [76]:
data.tail(30)

,testcase_id,word_num,word,label
15069,398,36,недопустимые,O
15070,398,37,значения,O
15071,398,38,параметров,O
15072,398,39,запроса,O
15074,399,0,менеджер,B-ACTOR
15075,399,1,проекта,I-ACTOR
15076,399,2,пытается,O
15077,399,3,изменить,O
15078,399,4,бюджет,O
15079,399,5,завершенного,O


In [77]:
X = data[["testcase_id","word"]]
Y = data["label"]

In [78]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

In [79]:
train_data = pd.DataFrame({"sentence_id":x_train["testcase_id"],"words":x_train["word"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["testcase_id"],"words":x_test["word"],"labels":y_test})

In [80]:
test_data

,sentence_id,words,labels
8569,220,ответ,O
5568,146,bad,O
2195,65,ошибки,O
270,9,сообщение,O
5220,139,новый,O
...,...,...,...
9270,240,получает,O
6893,176,сбоя,O
12671,335,запрос,O
10955,285,новом,O


In [81]:
labels = data["label"].unique().tolist()
labels

['B-ACTOR', 'O', 'B-REQUEST_TYPE', 'B-ENDPOINT', 'B-RESPONSE_CODE', 'I-ACTOR']

In [82]:
data["label"].value_counts()

O                  9165
B-REQUEST_TYPE      400
B-ENDPOINT          400
B-ACTOR             387
B-RESPONSE_CODE     200
I-ACTOR              77
Name: label, dtype: int64

In [83]:
# Define conditions
condition1 = data['label'] != 'O'
condition2 = (data.index % 20) == 0
data = data[condition1 | condition2]
data['label'].value_counts()

O                  463
B-REQUEST_TYPE     400
B-ENDPOINT         400
B-ACTOR            387
B-RESPONSE_CODE    200
I-ACTOR             77
Name: label, dtype: int64

## Build the Model

In [84]:
args = NERArgs()
args.num_train_epochs = 5
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [85]:
model = NERModel('bert', 'DeepPavlov/rubert-base-cased', labels=labels, args=args, use_cuda=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [86]:
model.train_model(train_data, eval_data=test_data, acc=balanced_accuracy_score)

  0%|          | 0/1 [00:00<?, ?it/s]

2024-04-24 20:34:25.739415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/13 [00:00<?, ?it/s]

(65, 0.2063508851023821)

In [87]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

2024-04-24 20:44:19.338849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [88]:
result

{'eval_loss': 0.04416786861391021,
 'precision': 0.9927007299270073,
 'recall': 0.951048951048951,
 'f1_score': 0.9714285714285715}

## Predictions

In [89]:
path_to_testcases = 'DATA/SYNTHETIC/synthetic_testcases.csv'
tc_df = pd.read_csv(path_to_testcases, sep=';', index_col=0)
tc_df.head()

,text,description,action,expected_response,positive,actor,request_type,endpoint,response_code
0,администратор удаляет товар из каталога.\nдейс...,администратор удаляет товар из каталога.,"delete-запрос к конечной точке ""/catalog/remov...",подтверждение удаления товара из каталога.,True,администратор,delete,/catalog/remove,200
1,модератор удаляет комментарий под постом.\nдей...,модератор удаляет комментарий под постом.,"delete-запрос к конечной точке ""/post/comment/...",подтверждение удаления комментария.,True,модератор,delete,/post/comment/remove,200
2,менеджер удаляет клиента из базы данных.\nдейс...,менеджер удаляет клиента из базы данных.,"delete-запрос к конечной точке ""/database/cust...",подтверждение удаления клиента из базы данных.,True,менеджер,delete,/database/customer/remove,200
3,редактор удаляет статью из журнала.\nдействие:...,редактор удаляет статью из журнала.,"delete-запрос к конечной точке ""/magazine/arti...",подтверждение удаления статьи из журнала.,True,редактор,delete,/magazine/article/remove,200
4,модератор удаляет изображение из галереи.\nдей...,модератор удаляет изображение из галереи.,"delete-запрос к конечной точке ""/gallery/image...",подтверждение удаления изображения из галереи.,True,модератор,delete,/gallery/image/remove,200


In [90]:
prediction, model_output = model.predict(tc_df.text[:100].tolist())

  0%|          | 0/1 [00:00<?, ?it/s]

2024-04-24 20:45:06.073639: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Running Prediction:   0%|          | 0/4 [00:00<?, ?it/s]

In [91]:
prediction

[[{'администратор': 'B-ACTOR'},
  {'удаляет': 'O'},
  {'товар': 'O'},
  {'из': 'O'},
  {'каталога.': 'O'},
  {'действие:': 'O'},
  {'delete-запрос': 'B-REQUEST_TYPE'},
  {'к': 'O'},
  {'конечной': 'O'},
  {'точке': 'O'},
  {'"/catalog/remove"': 'O'},
  {'с': 'O'},
  {'параметром': 'O'},
  {'product_id.': 'O'},
  {'ожидаемый': 'O'},
  {'ответ:': 'O'},
  {'подтверждение': 'O'},
  {'удаления': 'O'},
  {'товара': 'O'},
  {'из': 'O'},
  {'каталога.': 'O'}],
 [{'модератор': 'B-ACTOR'},
  {'удаляет': 'O'},
  {'комментарий': 'O'},
  {'под': 'O'},
  {'постом.': 'O'},
  {'действие:': 'O'},
  {'delete-запрос': 'B-REQUEST_TYPE'},
  {'к': 'O'},
  {'конечной': 'O'},
  {'точке': 'O'},
  {'"/post/comment/remove"': 'O'},
  {'с': 'O'},
  {'параметром': 'O'},
  {'comment_id.': 'O'},
  {'ожидаемый': 'O'},
  {'ответ:': 'O'},
  {'подтверждение': 'O'},
  {'удаления': 'O'},
  {'комментария.': 'O'}],
 [{'менеджер': 'B-ACTOR'},
  {'удаляет': 'O'},
  {'клиента': 'O'},
  {'из': 'O'},
  {'базы': 'O'},
  {'данных.'